# Extract target hydrophone click segments

This notebook extracts click segmented detected on target hydrophones for later computation of click attributes such as RL and SNR.

The click segnments are saved in `.npy` files under `hydrophone/extracted_clicks`.

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

import soundfile as sf

In [2]:
import pp_utils.file_handling as fd
import pp_utils.hydro_clicks as hc

## Set paths

In [3]:
analysis_path = Path('../data_processed/')
raw_path = Path('../data_raw/')

# hydrophone click detection folders
hydro_wav_path = analysis_path / 'hydrophone/wav_filtered/'
hydro_click_path = analysis_path / 'hydrophone/click_reclean_202103/points_cleaned_20211019/'

In [4]:
# click output folder
output_click_path = analysis_path / "hydrophone/extracted_clicks"

if not output_click_path.exists():
    output_click_path.mkdir()

Load dataframe with all info:

In [5]:
# main dataframe with all info
df_main = pd.read_csv(
    analysis_path / 'all_info_csv/main_info_append_09.csv',
    index_col=0
)

In [6]:
# filename prefix
df_main['fname_prefix'] = df_main.apply(
    lambda x: '%s_s%s_t%s' % ((x['DATE']), x['SESSION'], x['TRIAL']), axis=1
)

Get hydrophone sampling rate

In [7]:
# arbitrarily pick one file to get hydrophone sampling rate
hydro_raw_path = raw_path / "20190628/session2/target"
hydro_raw_file = hydro_raw_path / "trial_2_1.0V(peak)__28-06-2019_11_36_29.wav"

In [8]:
_, fs_hydro = sf.read(hydro_raw_file, 1)

## Extract clicks from all trials

In [9]:
window_len_sec = 256e-6  # window length to include
perc_before_pk = 50      # percentage of time window to shift before the detected peak

In [10]:
for trial_idx in df_main.index:

    fname_prefix = df_main.loc[trial_idx, "fname_prefix"]
    print(f"\n\ntrial {trial_idx}: {fname_prefix}")
    hydro_wav_file = list(hydro_wav_path.glob(f"t{trial_idx:03d}_*.wav"))
    
    # trial: 81: combine the 2 wav files
    if trial_idx == 81:
        hydro_wav_file_combine = hydro_wav_path / "t081_combine.wav"
        if not hydro_wav_file_combine.exists():
            data1, fs = sf.read(hydro_wav_file[0])
            data2, fs = sf.read(hydro_wav_file[1])
            sf.write(hydro_wav_file_combine, np.vstack((data1, data2)), fs)
        hydro_wav_file = hydro_wav_file_combine
    # all other trials: use the first hydro wav file
    else:
        hydro_wav_file = hydro_wav_file[0]
    
    print(f'  -- wav file:       {hydro_wav_file.name}')
    
    # load detection
    try:
        hydro_click_file = hydro_click_path / (f"t{trial_idx:03d}_hydrophone_cleaned.npz")
        df_ch0, df_ch1 = hc.load_detected_click_dfs(hydro_click_file)
        print(f'  -- detection file: {hydro_click_file.name}')
    except FileNotFoundError:
        print('No detection file, do not consider this trial')
        print('')
        print('')
        continue
        
    if len(df_ch0) == 0 or len(df_ch1) == 0:
        print('One channel not detected, do not consider this trial')
        print('')
        print('')
        continue
    
    # extract and gather clicks
    clicks_mtx0 = hc.gather_clicks(
        clk_file=hydro_wav_file, 
        df_clk=df_ch0, 
        fs_hydro=fs_hydro,
        ch=0,
        click_len_sec=window_len_sec, 
        perc_before_pk=perc_before_pk
    )
    clicks_mtx1 = hc.gather_clicks(
        clk_file=hydro_wav_file, 
        df_clk=df_ch1, 
        fs_hydro=fs_hydro,
        ch=1,
        click_len_sec=window_len_sec, 
        perc_before_pk=perc_before_pk
    )
    
    # save extracted clicks
    np.save(output_click_path / f"{fname_prefix}_extracted_clicks_ch0.npy", clicks_mtx0)
    np.save(output_click_path / f"{fname_prefix}_extracted_clicks_ch1.npy", clicks_mtx1)




trial 0: 20190625_s1_t0
  -- wav file:       t000_40dBgain_trial__0_1.0V(peak)__25-06-2019_10_19_28.wav
No detection file, do not consider this trial




trial 1: 20190625_s1_t0
  -- wav file:       t001_40dBgain_trial__0_1.0V(peak)__25-06-2019_10_19_45.wav
  -- detection file: t001_hydrophone_cleaned.npz
One channel not detected, do not consider this trial




trial 2: 20190625_s1_t1
  -- wav file:       t002_40dBgain_trial__1_1.0V(peak)__25-06-2019_10_21_55.wav
  -- detection file: t002_hydrophone_cleaned.npz




trial 3: 20190625_s1_t2
  -- wav file:       t003_40dBgain_trial__2_1.0V(peak)__25-06-2019_10_24_23.wav
  -- detection file: t003_hydrophone_cleaned.npz




trial 4: 20190625_s1_t3
  -- wav file:       t004_40dBgain_trial__3_1.0V(peak)__25-06-2019_10_25_54.wav
  -- detection file: t004_hydrophone_cleaned.npz




trial 5: 20190625_s1_t4
  -- wav file:       t005_40dBgain_trial__4_1.0V(peak)__25-06-2019_10_27_42.wav
  -- detection file: t005_hydrophone_cleaned.npz




trial 6: 20190625_s1_t5
  -- wav file:       t006_40dBgain_trial__5_1.0V(peak)__25-06-2019_10_29_52.wav
  -- detection file: t006_hydrophone_cleaned.npz


...



trial 242: 20190704_s3_t8
  -- wav file:       t242_trial_8_1.0V(peak)__04-07-2019_14_53_32.wav
  -- detection file: t242_hydrophone_cleaned.npz




trial 243: 20190704_s3_t9
  -- wav file:       t243_trial_9_1.0V(peak)__04-07-2019_14_54_32.wav
  -- detection file: t243_hydrophone_cleaned.npz




trial 244: 20190704_s3_t10
  -- wav file:       t244_trial_10_1.0V(peak)__04-07-2019_14_55_29.wav
  -- detection file: t244_hydrophone_cleaned.npz




trial 245: 20190704_s3_t11
  -- wav file:       t245_trial_11_1.0V(peak)__04-07-2019_14_56_14.wav
  -- detection file: t245_hydrophone_cleaned.npz




trial 246: 20190704_s3_t12
  -- wav file:       t246_trial_12_1.0V(peak)__04-07-2019_14_58_22.wav
  -- detection file: t246_hydrophone_cleaned.npz
